In [ ]:
!git clone https://github.com/AtharvaPawar456/District-HeatIndex-Predication-using-LSTM.git

In [ ]:
# Import necessary libraries
import pandas as pd

# Load the dataset
file_path = '/content/District-HeatIndex-Predication-using-LSTM/normalized_data.csv'
data = pd.read_csv(file_path)

# Describe the dataset
data.describe()

In [ ]:
data.head()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# # Load the dataset
# file_path = '/content/District-HeatIndex-Predication-using-LSTM/normalized_data.csv'
# data = pd.read_csv(file_path)

# Create a vocabulary list for the 'DISTRICT' column
district_vocab = list(data['DISTRICT'].unique())
district_index = {district: index for index, district in enumerate(district_vocab)}

# Map the 'DISTRICT' column to numerical indices
data['DISTRICT_INDEX'] = data['DISTRICT'].map(district_index)

# Define features and target
features = data.drop(['HEAT_INDEX', 'CATEGORY', 'DISTRICT'], axis=1)  # Exclude target, category, and original district columns

# target = data['CATEGORY']
target = data['HEAT_INDEX']

In [ ]:
features.head()

In [ ]:
features.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Concatenate features and target for correlation analysis
correlation_data = pd.concat([features, target], axis=1)

# Calculate the correlation matrix
correlation_matrix = correlation_data.corr()

# Plot the heatmap of the correlation matrix
plt.figure(figsize=(15, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix")
plt.show()

In [ ]:
# Get the correlation of each feature with the target variable
target_correlation = correlation_matrix['HEAT_INDEX'].abs().sort_values(ascending=False)

# Print the most correlated to least correlated features
print("Features sorted by correlation with CATEGORY:")
print(target_correlation)

# 3. LSTM with All Feature

selected_features = ['T2M', 'T_Fahrenheit', 'TS', 'T2M_MIN', 'T2M_MAX', 'T2MWET', 'WS10M_RANGE', 'WD50M', 'WD10M', 'WS10M_MAX', 'RH2M', 'MO', 'WS10M', 'T2MDEW', 'QV2M', 'WS50M', 'LON', 'DISTRICT_INDEX', 'LAT', 'PS', 'PRECTOTCORR', 'YEAR', 'T2M_RANGE', 'DY', 'WS10M_MIN']


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Select relevant features based on correlation results
selected_features = ['T2M', 'T_Fahrenheit', 'TS', 'T2M_MIN', 'T2M_MAX', 'T2MWET', 'WS10M_RANGE', 'WD50M', 'WD10M', 'WS10M_MAX', 'RH2M', 'MO', 'WS10M', 'T2MDEW', 'QV2M', 'WS50M', 'LON', 'DISTRICT_INDEX', 'LAT', 'PS', 'PRECTOTCORR', 'YEAR', 'T2M_RANGE', 'DY', 'WS10M_MIN']


# Use only the selected features and target variable for the model
selected_data = correlation_data[selected_features + ['HEAT_INDEX']]

# Define features and target
features = selected_data[selected_features]
target = selected_data['HEAT_INDEX']

# Normalize the features
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Reshape features for LSTM input (assuming each row is a time step)
features_reshaped = np.reshape(features_scaled, (features_scaled.shape[0], 1, features_scaled.shape[1]))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_reshaped, target, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='linear'))  # Output layer for regression with linear activation

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=6, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {round(accuracy * 100, 2)} %')

In [ ]:
import pickle

# Save the model
model.save('lstm_model_all_feature.h5')

# Save the district_index
with open('district_index.pkl', 'wb') as f:
    pickle.dump(district_index, f)

# Save the input feature name list
feature_names = features.columns.tolist()
with open('feature_names_all_feature.pkl', 'wb') as f:
    pickle.dump(feature_names, f)

In [ ]:
# selected_features = ['T2M', 'RH2M', 'T2MDEW', 'WD10M', 'HEAT_INDEX']
__selected_features = ['T2M', 'T_Fahrenheit', 'TS', 'T2M_MIN', 'T2M_MAX', 'T2MWET', 'WS10M_RANGE', 'WD50M', 'WD10M', 'WS10M_MAX', 'RH2M', 'MO', 'WS10M', 'T2MDEW', 'QV2M', 'WS50M', 'LON', 'DISTRICT_INDEX', 'LAT', 'PS', 'PRECTOTCORR', 'YEAR', 'T2M_RANGE', 'DY', 'WS10M_MIN']


# Assuming your DataFrame is named df
new_data_for_inference = data.loc[10, __selected_features]

print("Data for the specified features in the first row:")
print(new_data_for_inference)

In [ ]:
import pickle
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('lstm_model_all_feature.h5')

# Load the district_index
with open('district_index.pkl', 'rb') as f:
    loaded_district_index = pickle.load(f)

# Load the input feature name list
with open('feature_names_all_feature.pkl', 'rb') as f:
    loaded_feature_names = pickle.load(f)


# selected_features = ['T2M', 'T_Fahrenheit', 'TS', 'T2M_MIN', 'T2M_MAX', 'T2MWET', 'WS10M_RANGE', 'WD50M', 'WD10M', 'WS10M_MAX', 'RH2M', 'MO', 'WS10M', 'T2MDEW', 'QV2M', 'WS50M', 'LON', 'DISTRICT_INDEX', 'LAT', 'PS', 'PRECTOTCORR', 'YEAR', 'T2M_RANGE', 'DY', 'WS10M_MIN']

new_data_for_inference = pd.DataFrame({
'T2M'             :  [0.325153],
'T_Fahrenheit'       :  [72.91],
'TS'              :  [0.336126],
'T2M_MIN'         :  [0.431715],
'T2M_MAX'         :  [0.305014],
'T2MWET'          :  [0.631997],
'WS10M_RANGE'     :  [0.164191],
'WD50M'           :  [0.609584],
'WD10M'           :  [0.614839],
'WS10M_MAX'       :  [0.175806],
'RH2M'            :  [0.733267],
'MO'    :                   [1],
'WS10M'           :  [0.176881],
'T2MDEW'          :  [0.766538],
'QV2M'            :  [0.549058],
'WS50M'           :  [0.208062],
'LON'               :  [79.293],
'DISTRICT_INDEX'    :      [10],
'LAT'               :  [19.366],
'PS'               :  [0.43377],
'PRECTOTCORR'     :  [0.031997],
'YEAR'            :      [2001],
'T2M_RANGE'       :  [0.367852],
'DY'              :         [1],
'WS10M_MIN'       :  [0.139295],
})




# Normalize the features using the previously fitted scaler
new_data_scaled = scaler.transform(new_data_for_inference)

# Reshape features for LSTM input
new_data_reshaped = np.reshape(new_data_scaled, (new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))

# Make predictions for HEAT_INDEX
heat_index_prediction = model.predict(new_data_reshaped)

pred_Heat_index = round(heat_index_prediction.flatten()[0], 2)

# Print predicted HEAT_INDEX
print("Predicted HEAT_INDEX:", pred_Heat_index)

In [ ]:
# Assuming you have the true HEAT_INDEX value for comparison
true_heat_index =  74.71      # Provide the true value here

# Calculate Mean Absolute Error (MAE)
mae = round(np.mean(np.abs(pred_Heat_index - true_heat_index)), 6)

# Print predicted HEAT_INDEX and MAE
print('''
# 3. LSTM with All Feature

selected_features = ['T2M', 'T_Fahrenheit', 'TS', 'T2M_MIN', 'T2M_MAX', 'T2MWET', 'WS10M_RANGE', 'WD50M', 'WD10M', 'WS10M_MAX', 'RH2M', 'MO', 'WS10M', 'T2MDEW', 'QV2M', 'WS50M', 'LON', 'DISTRICT_INDEX', 'LAT', 'PS', 'PRECTOTCORR', 'YEAR', 'T2M_RANGE', 'DY', 'WS10M_MIN']

''')
print("Predicted HEAT_INDEX:\t", pred_Heat_index)
print("Actual HEAT_INDEX: \t", true_heat_index)
print("\nMean Absolute Error:", mae , "%")

# Output:

Predicted HEAT_INDEX:	 73.8

Actual HEAT_INDEX: 	 74.71

Mean Absolute Error: 0.909997 %